Naivni algoritam:

In [1]:
import numpy as np
from numpy import linalg as la 

In [2]:
def koeficijenti(matrica):
#   resenje sistema oblika: A * x = b, dakle x = A^-1 * b
    A = matrica[0:3]
    A = np.transpose(A)
    b = matrica[3]
    
    A_inv = la.inv(A)
    x = np.matmul(A_inv, b)
    
    return x
    

In [3]:
def naivni_algoritam(original, slika):

    p1 = koeficijenti(original)
    p2 = koeficijenti(slika)

# p1 je matrica sa kolonama alfa * A, beta * B i gama * C
    p1 = [[p1[0] * x for x in original[0]],
          [p1[1] * x for x in original[1]],
          [p1[2] * x for x in original[2]]
        ]
    p1 = np.transpose(p1)
    
# p2 je matrica sa kolonama alfa' * A', beta' * B' i gama' * C'
    p2 = [[p2[0] * x for x in slika[0]],
          [p2[1] * x for x in slika[1]],
          [p2[2] * x for x in slika[2]]
        ]

    p2 = np.transpose(p2)

# trazena matrica preslikavanja je p = p2*p1^-1
    p = np.matmul(p2,la.inv(p1))

    return p

In [4]:
originali = [[-3, -1, 1],
             [3, -1, 1], 
             [1, 1, 1], 
             [-1, 1, 1]
            ]

slike = [[-2, -1, 1],
         [2, -1, 1],
         [2, 1, 1], 
         [-2, 1, 1]
        ]

P1 = naivni_algoritam(originali, slike)
P1 = np.round(P1, decimals=5)
print("Matrica projektivnog preslikavanja dobijena naivnim algoritmom:\n")
print(P1)

Matrica projektivnog preslikavanja dobijena naivnim algoritmom:

[[ 2.  0.  0.]
 [ 0.  2. -1.]
 [ 0. -1.  2.]]


DLT algoritam:

In [5]:
def matrica_korespodencije(o, s):
    m = np.matrix([[0, 0, 0, -s[2]*o[0], -s[2]*o[1], -s[2]*o[2], s[1]*o[0], s[1]*o[1], s[1]*o[2]],
     [s[2]*o[0], s[2]*o[1], s[2]*o[2], 0, 0, 0, -s[0]*o[0], -s[0]*o[1], -s[0]*o[2]]])

    return m

In [6]:

def dlt_algoritam(n, originali, slike):
    
    matrica = matrica_korespodencije(originali[0], slike[0])
    
    for i in range(1, n):
        m = matrica_korespodencije(originali[i], slike[i])
        matrica = np.concatenate((matrica, m), axis = 0)
    
    U, D, Vt = la.svd(matrica, full_matrices=True)
    
# poslednja kolona matrice V, tj. poslednja vrsta matrice Vt
    P = Vt[-1]
    P = P.reshape(3,3)

    return P

In [7]:
originali = [[-3, -1, 1],
             [3, -1, 1], 
             [1, 1, 1], 
             [-1, 1, 1]
            ]

slike = [[-2, -1, 1],
         [2, -1, 1],
         [2, 1, 1], 
         [-2, 1, 1]
        ]

#Pokrecemo DLT algoritam za 6 tacaka    
P2 = dlt_algoritam(4, originali, slike)
P2 = np.round(P2,decimals=10)

print("Matrica projektivnog preslikavanja dobijena dlt algoritmom")
print(P2, "\n")

Matrica projektivnog preslikavanja dobijena dlt algoritmom
[[ 0.53452248  0.         -0.        ]
 [ 0.          0.53452248 -0.26726124]
 [-0.         -0.26726124  0.53452248]] 



In [8]:
print((P2 / P2[0][0]) * P1[0][0])

[[ 2.  0. -0.]
 [ 0.  2. -1.]
 [-0. -1.  2.]]


Normalizovani DLT algoritam:

In [9]:
def afina(tacka):
    return [tacka[0]/tacka[2], tacka[1]/tacka[2]]

def homogena(tacka):
    return [tacka[0], tacka[1], 1]


In [10]:
def normalizacija(n, tacke):
    afine_tacke = []

# teziste tacaka: 

    cx = 0
    cy = 0
    
    for i in range(n):
        afine_tacke.append(afina(tacke[i]))
        cx += afine_tacke[i][0]
        cy += afine_tacke[i][1]
    
    cx, cy = cx/n, cy/n
    
# matrica translacije tezista u koordinatni pocetak:
    G = [[1,0,-cx], [0,1,-cy], [0,0,1]]

    for i in range(n):
        afine_tacke[i] = np.matmul(G, homogena(afine_tacke[i]))

# skaliranje tacaka tako da prosecno rastojanje od koordinatnog pocetka bude sqrt(2)
    dist = 0
    for i in range(n):
        dist += np.sqrt(np.square(afine_tacke[i][0]) + np.square(afine_tacke[i][1]))

    dist = dist/n

    S = [[np.sqrt(2)/dist,0, 0], [0,np.sqrt(2)/dist,0], [0,0,1]]

    for i in range(n):
        afine_tacke[i] = np.matmul(S, homogena(afine_tacke[i]))


# matrica normalizacije: T = SG    
    T = np.matmul(S,G)

    return T

In [11]:
def primeni(tacke, matrica):
    n = len(tacke)
    nove_t = []
    for i in range(n):
         nove_t.append(np.matmul(matrica, tacke[i]))
    return nove_t

In [12]:
def normalizovani_dlt(n,originali, slike):

# matrice normalizacije:
    originali_mat = normalizacija(n, originali)
    slike_mat = normalizacija(n, slike)

# normalizovane tacke:
    originali_n = primeni(originali, originali_mat)
    slike_n = primeni(slike, slike_mat)

# matrica P' se dobija primenom obicnog DLT algoritma na normalizovane tacke
    matrica_p = dlt_algoritam(n, originali_n, slike_n)

# trazena matrica je T'^-1 * P * T  
    matrica = np.matmul(matrica_p, originali_mat)
    matrica = np.matmul(la.inv(slike_mat), matrica)

    return np.round(matrica, decimals=10)

In [13]:
originali = [[-3, -1, 1],
             [3, -1, 1], 
             [1, 1, 1], 
             [-1, 1, 1]
            ]

slike = [[-2, -1, 1],
         [2, -1, 1],
         [2, 1, 1], 
         [-2, 1, 1]
        ]

matrica = normalizovani_dlt(4, originali, slike)
print("Matrica projektivnog preslikavanja dobijena normalizovanim dlt algoritmom\n")
print(matrica, "\n")

Matrica projektivnog preslikavanja dobijena normalizovanim dlt algoritmom

[[ 0.50971765 -0.         -0.        ]
 [ 0.          0.50971765 -0.25485883]
 [ 0.         -0.25485883  0.50971765]] 



Testiranje na sopstvenim primerima:

In [14]:
originali = [[0, 0, 1],
             [1, 2, 1], 
             [4, 2, 1], 
             [5, 0, 1]
            ]

slike = [[0, 1, 1],
         [0, 3, 1],
         [4, 3, 1], 
         [4, 1, 1]
        ]


In [15]:
# matrica dobijena naivnim algoritmom
M1 = naivni_algoritam(originali, slike)
M1 = np.round(M1, decimals = 5)
print(M1)

[[ 0.8 -0.4  0. ]
 [ 0.   0.4  1. ]
 [ 0.  -0.2  1. ]]


In [16]:
# matrica dobijena dlt algoritmom
M2 = dlt_algoritam(4, originali, slike)
M2 = np.round(M2, decimals = 5)
print(M2)

[[ 0.46188 -0.23094 -0.     ]
 [ 0.       0.23094  0.57735]
 [ 0.      -0.11547  0.57735]]


In [17]:
# provera da li su matrice dobijene naivnim i dlt algoritmom proporcionalne
(M2 / M2[0][0])  * M1[0][0]

array([[ 0.8, -0.4, -0. ],
       [ 0. ,  0.4,  1. ],
       [ 0. , -0.2,  1. ]])

In [18]:
# matrica dobijena normalizovanim dlt algoritmom
M3 = normalizovani_dlt(4, originali, slike)
M3 = np.round(M3, decimals = 5)
print(M3)

[[ 0.55869 -0.27935  0.     ]
 [ 0.       0.27935  0.69837]
 [ 0.      -0.13967  0.69837]]


In [19]:
# poredjenje matrica dobijenih klasicnim i normalizovanim dlt algoritmom
# matrice se poklapaju na prvih 5 decimala (na toliko decimala je i izvrseno zaokruzivanje)
(M3 / M3[0][0])  * M2[0][0]

array([[ 0.46188   , -0.23094413,  0.        ],
       [ 0.        ,  0.23094413,  0.5773562 ],
       [ 0.        , -0.11546793,  0.5773562 ]])

In [20]:
# dodavanje novih tacaka za testiranje dlt algoritma za vise od 4 korespodencije
# odabrane tacke su sredista osnovica trapeza, njihove slike su dobijene naivnim algoritmom
t1 = [2.5, 0, 1]
t1p = np.matmul(M1, np.transpose(t1))
t1p = np.transpose(t1p / t1p[2])

t2 = [2.5, 2, 1]
t2p = np.matmul(M1, np.transpose(t2))
t2p = np.transpose(t2p / t2p[2])

print(t1, t1p, t2, t2p)

originali.append(t1)
originali.append(t2)
slike.append(t1p)
slike.append(t2p)

[2.5, 0, 1] [2. 1. 1.] [2.5, 2, 1] [2. 3. 1.]


In [21]:
originali

[[0, 0, 1], [1, 2, 1], [4, 2, 1], [5, 0, 1], [2.5, 0, 1], [2.5, 2, 1]]

In [22]:
slike

[[0, 1, 1],
 [0, 3, 1],
 [4, 3, 1],
 [4, 1, 1],
 array([2., 1., 1.]),
 array([2., 3., 1.])]

In [23]:
# matrica dobijena dlt algoritmom za 6 korespodencija
M4 = dlt_algoritam(6, originali, slike)
M4 = np.round(M4, decimals = 5)
print(M4)

[[-0.46188  0.23094 -0.     ]
 [ 0.      -0.23094 -0.57735]
 [ 0.       0.11547 -0.57735]]


In [24]:
# ova matrica odredjuje isto preslikavanje kao i matrica dobijena naivnim algoritmom
(M4 / M4[0][0])  * M1[0][0]

array([[ 0.8, -0.4,  0. ],
       [-0. ,  0.4,  1. ],
       [-0. , -0.2,  1. ]])

In [25]:
# matrica dobijena normalizovanim dlt algoritmom za 6 korespodenciija
M5 = normalizovani_dlt(6, originali, slike)
M5 = np.round(M5, decimals = 5)
print(M5)

[[ 0.56259 -0.2813   0.     ]
 [-0.       0.2813   0.70324]
 [-0.      -0.14065  0.70324]]


In [26]:
# poredjenje te dve matrice
# ponovo se poklapaju na prvih 5 decimala, mala je greska, pa moze da se kaze da odredjuju isto preslikavanje
(M5 / M5[0][0])  * M4[0][0]

array([[-0.46188   ,  0.2309441 , -0.        ],
       [ 0.        , -0.2309441 , -0.57735205],
       [ 0.        ,  0.11547205, -0.57735205]])

In [27]:
# transliramo sve koordinate za vektor (2, 3)
translacija = [[1, 0, 2], [0, 1, 3], [0, 0, 1]]

originali_t = primeni(originali, translacija)
slike_t = primeni(slike, translacija)

In [28]:
# matrica dobijena normalizovanim dlt na transiranim tackama
M6 = normalizovani_dlt(6, originali_t, slike_t)
M6 = np.round(M6, decimals = 5)
print(M6)

[[ 0.56259 -0.56259  1.96908]
 [ 0.      -0.14065  3.23492]
 [ 0.      -0.14065  1.12519]]


In [29]:
# vracanje na stari koordinatni sistem
# primetimo da se dobije ista matrica kao za pocetne tacke
#  to je i ocekivano jer je normalizovani dlt invarijantan u odnosu na promenu koordinata
M7 = np.matmul(la.inv(translacija), M6)
M7 = np.matmul(M7, translacija)
M7 = np.round(M7, decimals = 5)
print(M7)
print(M5)

[[ 5.6259e-01 -2.8129e-01  1.0000e-05]
 [ 0.0000e+00  2.8130e-01  7.0325e-01]
 [ 0.0000e+00 -1.4065e-01  7.0324e-01]]
[[ 0.56259 -0.2813   0.     ]
 [-0.       0.2813   0.70324]
 [-0.      -0.14065  0.70324]]
